In [1]:
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(['localhost'], port=9042) # You can specify multiple contact points if needed
session = cluster.connect() 

In [19]:
session.execute('''CREATE KEYSPACE IF NOT EXISTS cbd WITH replication 
                = {'class': 'SimpleStrategy', 'replication_factor': 1}''')


In [20]:
session.execute('''
CREATE TABLE IF NOT EXISTS cbd.inscritos (
    codigo_estabelecimento INT,
    estabelecimento TEXT,
    codigo_unidade_organica INT,
    unidade_organica TEXT,
    nuts_ii_2002 TEXT,
    nuts_iii_2002 TEXT,
    nuts_ii_2013 TEXT,
    nuts_iii_2013 TEXT,
    distrito TEXT,
    concelho TEXT,
    natureza_estabelecimento_ensino TEXT,
    tipo_ensino TEXT,
    codigo_curso INT,
    curso TEXT,
    curso_ciclo_estudos TEXT,
    area_educacao_formacao_codigo_area_geral INT,
    area_educacao_formacao_area_geral TEXT,
    area_educacao_formacao_codigo_area_especifica INT,
    area_educacao_formacao_area_especifica TEXT,
    area_educacao_formacao_codigo_area_detalhada INT,
    area_educacao_formacao_area_detalhada TEXT,
    sexo TEXT,
    nacionalidade TEXT,
    estudante_trabalhador TEXT,
    primeira_vez TEXT,
    tempo_parcial TEXT,
    bolseiro TEXT,
    total INT,
    PRIMARY KEY (codigo_estabelecimento)
);
''')

In [9]:
import pandas as pd

C:\Users\bruno\AppData\Local\Temp\ipykernel_29308\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
df = pd.read_excel(r'C:\Users\bruno\Desktop\Projeto cbd\Inscritos\Inscritos_2016_2017.xlsx')

In [27]:
print(df.dtypes)

Código Estabelecimento                                   object
Estabelecimento                                          object
Código Unidade Orgânica                                 float64
Unidade Orgânica                                         object
NUTS II - 2002                                           object
NUTS III - 2002                                          object
NUTS II - 2013                                           object
NUTS III - 2013                                          object
Distrito                                                 object
Concelho                                                 object
Natureza do Estabelecimento de Ensino                    object
Tipo de Ensino                                           object
Código Curso                                             object
Curso                                                    object
Curso/Ciclo de Estudos                                   object
Área de Educação e Formação - Código Áre

In [29]:
df.columns

Index(['Código Estabelecimento', 'Estabelecimento', 'Código Unidade Orgânica',
       'Unidade Orgânica', 'NUTS II - 2002', 'NUTS III - 2002',
       'NUTS II - 2013', 'NUTS III - 2013', 'Distrito', 'Concelho',
       'Natureza do Estabelecimento de Ensino', 'Tipo de Ensino',
       'Código Curso', 'Curso', 'Curso/Ciclo de Estudos',
       'Área de Educação e Formação - Código Área Geral',
       'Área de Educação e Formação - Área Geral',
       'Área de Educação e Formação - Código Área Específica',
       'Área de Educação e Formação - Área Específica',
       'Área de Educação e Formação - Código Área Detalhada',
       'Área de Educação e Formação - Área Detalhada', 'Sexo', 'Nacionalidade',
       'Estudante Trabalhador', 'Primeira Vez', 'Tempo Parcial', 'Bolseiro',
       'Total'],
      dtype='object')

In [31]:
df2 = df[['Código Estabelecimento','Estabelecimento','Código Curso','Curso']]

In [16]:
session.execute("DROP TABLE IF EXISTS cbd.inscritos")

In [17]:
session.execute('''
    CREATE TABLE cbd.inscritos (
    natureza_do_estabelecimento VARCHAR,
    nome_estabelecimento VARCHAR,
    tipo_de_ensino VARCHAR,
    nivel_de_formacao VARCHAR,
    curso VARCHAR,
    distrito VARCHAR,
    sexo VARCHAR,
    total INT,
    grande_grupo VARCHAR,
    pais VARCHAR,
    ano INT,
    PRIMARY KEY (natureza_do_estabelecimento, nome_estabelecimento, tipo_de_ensino, nivel_de_formacao, curso, ano)
);

''')

In [81]:
session.execute("DROP TABLE IF EXISTS cbd.teste")

In [61]:
a = df2.loc[0].to_list()
t = ','.join(["'" + item + "'" for item in a])
t

"'0100','Universidade dos Açores','5753','Ciências Agrárias'"

In [ ]:
for index, row in df2.iterrows():
    print(type(row['Código Estabelecimento']))

In [84]:
for index, row in df2.iterrows():
    session.execute(f'''INSERT INTO cbd.teste (
                        cod_estab,
                        estab,
                        cod_curso,
                        curso)
                        VALUES ({int(row['Código Estabelecimento'])}, '{row['Estabelecimento']}', {(row['Código Curso'])}, '{row['Curso']}'
                        )'''
                    )





InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid INTEGER constant (5753) for "cod_curso" of type text"

In [3]:
import glob
import pandas as pd

In [10]:
# diplomados
path = r"C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos"
excel_files = glob.glob(path + '/*.xlsx')

for file_path in excel_files:
    column_names = pd.read_excel(file_path, nrows=0).columns.tolist()

    print(column_names)

['Código Estabelecimento ', 'Estabelecimento', 'Código Unidade Orgânica', 'Unidade Orgânica', 'NUTS II', 'NUTS III', 'Distrito', 'Concelho', 'Subsistema de Ensino', 'Tipo de Ensino', 'Código Curso', 'Curso', 'Nivel de Formação', 'CNAEF - Código Grande Grupo', 'CNAEF - Grande grupo', 'CNAEF - Código Área de Estudo', 'CNAEF - Área de Estudo', 'CNAEF - Código Área de Educação e Formação ', 'CNAEF - Área de Educação e Formação', 'Sexo', 'Total']
['Código Estabelecimento ', 'Estabelecimento', 'Código Unidade Orgânica', 'Unidade Orgânica', 'NUTS II', 'NUTS III', 'Distrito', 'Concelho', 'Subsistema de Ensino', 'Tipo de Ensino', 'Código Curso', 'Curso', 'Nivel de Formação', 'CNAEF - Código Grande Grupo', 'CNAEF - Grande grupo', 'CNAEF - Código Área de Estudo', 'CNAEF - Área de Estudo', 'CNAEF - Código Área de Educação e Formação ', 'CNAEF - Área de Educação e Formação', 'Sexo', 'Total']
['Código Estabelecimento ', 'Estabelecimento', 'Código Unidade Orgânica', 'Unidade Orgânica', 'NUTS II', 'NU

In [8]:
import glob
import pandas as pd
import os

dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nivel de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande grupo':'grande_grupo',
                'País de Nacionalidade':'pais'}


path = r"C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos"

excel_files = glob.glob(path + '/*.xlsx')

script_dir = os.getcwd()

save_path = os.path.join(script_dir, 'datasets_inscritos')

os.makedirs(save_path, exist_ok=True)

for idx, file in enumerate(excel_files, 1):
    # Create an ExcelFile object without reading the file into memory
    xls = pd.ExcelFile(file)
    
    # Initialize a dictionary to store selected and renamed columns
    selected_columns = {}
    
    # Iterate over the sheet names in the Excel file
    for sheet_name in xls.sheet_names:
        # Read only the first row of each sheet to get the column names
        df_temp = pd.read_excel(xls, sheet_name, nrows=1)
        
        # Iterate over the columns in the DataFrame and check if they exist in the dictionary
        for col in df_temp.columns:
            if col in dict_colunas:
                # If the column exists in both the DataFrame and the dictionary, add it to the dictionary
                selected_columns[col] = dict_colunas[col]
    
    # If there are no matching columns, print a message and continue to the next file
    if not selected_columns:
        print(f"No matching columns found in file {file}, skipping...")
        continue
    
    # Select only the specified columns from each sheet and rename them
    for sheet_name in xls.sheet_names:
        df_selected = pd.read_excel(xls, sheet_name, usecols=selected_columns.keys())
        df_selected = df_selected.rename(columns=selected_columns)
        
        # Create the new filename
        new_filename = os.path.join(save_path, f"inscritos{idx}_{sheet_name}.xlsx")
        
        # Save the DataFrame to a new Excel file in the output directory
        df_selected.to_excel(new_filename, index=False)


KeyboardInterrupt: 

In [ ]:
session.execute('''
    CREATE TABLE cbd.inscritos (
    natureza_do_estabelecimento VARCHAR,
    nome_estabelecimento VARCHAR,
    tipo_de_ensino VARCHAR,
    nivel_de_formacao VARCHAR,
    curso VARCHAR,
    distrito VARCHAR,
    sexo VARCHAR,
    total INT,
    grande_grupo VARCHAR,
    pais VARCHAR,
    PRIMARY KEY (natureza_do_estabelecimento, nome_estabelecimento, tipo_de_ensino, nivel_de_formacao, curso)
);

''')

In [ ]:
for index, row in df2.iterrows():
    session.execute(f'''INSERT INTO cbd.teste (
                        cod_estab,
                        estab,
                        cod_curso,
                        curso)
                        VALUES ({int(row['Código Estabelecimento'])}, '{row['Estabelecimento']}', {(row['Código Curso'])}, '{row['Curso']}'
                        )'''
                    )
    

session.execute(f'''INSERT INTO cbd.inscritos(
                

)
''')

In [24]:
dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nivel de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande grupo':'grande_grupo',
                'País de Nacionalidade':'pais'}
selected_columns = {}

In [26]:
df_temp = pd.read_excel(r"C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos\Inscritos_2004_2005.xlsx",nrows=1)

for col in df_temp.columns:
        if col in dict_colunas:
            selected_columns[col] = dict_colunas[col]
column_names = df.columns.tolist()
df_selected = pd.read_excel(xls, sheet_name, usecols=selected_columns.keys())
df_selected = df_selected.rename(columns=selected_columns)


ValueError: Invalid file path or buffer object type: <class 'openpyxl.workbook.workbook.Workbook'>